**Import**

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

!pip install transformers
!pip install -U datasets
!pip install setfit

from collections import defaultdict, Counter
import json
import torch

from matplotlib import pyplot as plt

from datasets import load_dataset, DatasetDict

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Training and Validation**

In [ ]:
banktrak_df = pd.read_csv('/content/gdrive/MyDrive/Group 1: DSSI Summer 2025/Data/word_count.csv')
banktrak_df = banktrak_df[['text', 'contains_debt_instrument_information']]
train_df, val_df = train_test_split(banktrak_df, test_size=0.2, random_state=42)
dataset = DatasetDict({'train': Dataset.from_pandas(train_df), 'validation': Dataset.from_pandas(val_df)}) #use dataset dict to turn to dataframe to dict
dataset = dataset.rename_column("contains_debt_instrument_information", "label")
dataset = dataset.remove_columns("__index_level_0__")

train_dict = {
    "text": list(dataset["train"]["text"]),
    "label": list(dataset["train"]["label"])
}
val_dict = {
    "text": list(dataset["validation"]["text"]),
    "label": list(dataset["validation"]["label"])
}

dataset_fixed = DatasetDict({
    "train": Dataset.from_dict(train_dict),
    "validation": Dataset.from_dict(val_dict)
})

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 176
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 44
    })
})

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from setfit import SetFitModel, SetFitTrainer
from datasets import load_dataset
import torch
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5", use_differentiable_head=True).to(device)

body_params = model.model_body.parameters()
head_params = model.model_head.parameters()

from setfit import SetFitTrainer

trainer = SetFitTrainer(
    model=model,
    train_dataset=dataset_fixed['train'],    # use dataset_fixed here
    eval_dataset=dataset_fixed['validation'],  # use dataset_fixed here
    metric="accuracy",
    batch_size=8,
    num_epochs=3,
    column_mapping={"text": "text", "label": "label"},
    num_iterations=20
)

trainer.train()

# Now manually compute metrics
preds = trainer.model.predict(dataset['validation']['text'])
true = dataset['validation']['label']

precision, recall, f1, _ = precision_recall_fscore_support(true, preds, average="binary")
acc = accuracy_score(true, preds)

print(f"Accuracy: {acc}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

model_path = "/content/gdrive/MyDrive/DSSI/distilbert-model"
model.save_pretrained(model_path)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
/tmp/ipython-input-59-3016553422.py:52: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


AttributeError: 'Column' object has no attribute 'sort'

**Evaluate your model on Test Data**

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

eval_dataloader = DataLoader(tokenized_dataset['validation'], batch_size=len(tokenized_dataset['validation'])) #why dataloader again?
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #why this again, exaclty same. i feel dont need

model.eval()            #evalaution mode
test_batch_logits = []    #store models raw outputs logits
y_true = []                # stores groudn turth label from validaton set??
for batch_i, batch in enumerate(eval_dataloader): #go through batches
    with torch.no_grad():                          #clear graidnet
        # copy input to device
        input_ids = batch['input_ids'].to(device)            #add to deviice
        attention_mask = batch['attention_mask'].to(device)  #add device
        labels = batch['labels'].cpu().detach().numpy()      #removes labels

        # Call the model on test data
        output = model(input_ids = input_ids, attention_mask = attention_mask, labels =None) #labels none becuase didnt pass any
        test_batch_logits.append(output.logits)                #append output logits --stores logits before soft max
        y_true.extend(labels)                                  #add ground turth labels

print(len(test_batch_logits),len(eval_dataloader))      #shape of the final logits tensor ([num_examples, num_classes])
test_logits = torch.cat(test_batch_logits, dim=0)     #concatenates the logits from all batches along dimension 0.

#sanity check -> dimension 0 of your logits tensor should be same as the size of the test dataset
print(test_logits.shape,len(tokenized_dataset['validation']),len(y_true))

In [ ]:
#Convert the logits to predicted labels
y_pred = torch.argmax(test_logits, dim = 1).cpu().numpy()

print(y_true[:10])
print(y_pred[:10])

#sanity check: should have as many predictions as labels
assert len(y_pred)==len(y_true)

In [ ]:
# call the f1_score function
print('F1 Score:',f1_score(y_true, y_pred, average='binary')) #why add average binary?

# call the accuracy_score function
print('Accuracy Score:',accuracy_score(y_true, y_pred))

new stuff

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

#cm = confusion_matrix(y_true, y_pred)
#print("Confusion Matrix:\n", cm)

import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

In [ ]:
# pip install transformer-ranker

In [ ]:
# from datasets import load_dataset
# from transformer_ranker import TransformerRanker, prepare_popular_models

# # Step 1: Load the CoNLL-03 dataset from HuggingFace
# dataset = load_dataset('conll2003')

# # Step 2: Use our list of 17 'base' LMs as candidates
# language_models = prepare_popular_models('base')

# # Step 3: Initialize the ranker with the dataset
# ranker = TransformerRanker(dataset, dataset_downsample=0.2)

# # ... and run the ranker to obtain the ranking
# results = ranker.run(language_models, batch_size=64)

# # print the ranking
# print(results)